In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train.csv",dtype = {'Upc': str, 'FinelineNumber':str, 'TripType':"category" , 'Weekday':"category"})
test = pd.read_csv("test.csv",dtype = {'Upc': str, 'FinelineNumber':str, 'TripType':"category" , 'Weekday':"category"})

In [4]:
all_data = pd.concat([train,test])
all_data.reset_index(inplace=True,drop=True)

In [5]:
all_data.drop(['TripType', 'Upc'],axis = 1,inplace = True)

In [6]:
all_data['FinelineNumber_copy'] = all_data['FinelineNumber']

In [7]:
group_visit_fineline = all_data.groupby(['VisitNumber','FinelineNumber'],as_index=False).agg({'FinelineNumber_copy':'count'})

In [ ]:
# day= pd.get_dummies(all_data["VisitNumber", "Weekday"], columns = ["Weekday"])
# # day.drop_duplicates()

In [8]:
dictVector = []
visitnum = group_visit_fineline['VisitNumber'][1]
visitdict = {}
for i in range(len(group_visit_fineline)):
    if(group_visit_fineline['VisitNumber'][i] == visitnum):
        visitdict[group_visit_fineline['FinelineNumber'][i]] = group_visit_fineline['FinelineNumber_copy'][i]
    else:
        dictVector.append(visitdict)
        visitdict = {}
        visitdict[group_visit_fineline['FinelineNumber'][i]] = group_visit_fineline['FinelineNumber_copy'][i]
        visitnum = group_visit_fineline['VisitNumber'][i]
# last one
dictVector.append(visitdict)

In [9]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)

In [10]:
X = v.fit_transform(dictVector)

In [11]:
day = pd.get_dummies(all_data[["VisitNumber", "Weekday"]], columns = ["Weekday"])
day = day.drop_duplicates()

# find unique VisitNumber
uids = group_visit_fineline['VisitNumber'].unique()

# keep VisitNumber in uids
day = day[day['VisitNumber'].isin(uids)]

In [12]:
day.sort_values(['VisitNumber'], inplace=True)

In [13]:
new_X = np.concatenate( (X, day), axis=1)

In [16]:
new_X.shape

(188535, 5361)

In [33]:
dpt_no_dup = all_data[['VisitNumber','DepartmentDescription']].drop_duplicates()

In [34]:
dept = pd.get_dummies(dpt_no_dup, columns = ['DepartmentDescription'])

In [35]:
dept.sort_values(['VisitNumber'], inplace=True)

In [48]:
t = all_data.groupby(['VisitNumber','DepartmentDescription']).agg({'DepartmentDescription':'count'})

In [51]:
t1 = t.unstack()

In [56]:
t1 = t1['DepartmentDescription']

In [82]:
t1.fillna(0, inplace= True)

In [89]:
t1

DepartmentDescription,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
dp = []
for i in uids:
    dp.append(list(t1.loc[i]))

In [85]:
department = np.array(dp)

In [91]:
department.shape

(188535, 68)

In [92]:
department[department > 0] = 1

In [94]:
final = np.concatenate((new_X, department), axis=1)

In [96]:
final.shape

(188535, 5429)